# 1. Project Packages

In [15]:
from operator import truediv
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from torchsummary import summary
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
import scipy.io as sio
import torch.nn as nn
import torchvision
import numpy as np
import spectral
import torch
import math
import wget
import time
import os

# 2. Proposed Model

In [16]:
class FE_HybridSN(nn.Module):
    def __init__(self):
        super(FE_HybridSN, self).__init__()
        self.conv3d_1 = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=(7, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(8),
            nn.ReLU(inplace=True),
        )
        self.conv3d_2 = nn.Sequential(
            nn.Conv3d(8, 16, kernel_size=(5, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(16),
            nn.ReLU(inplace=True),
        )
        self.conv3d_3 = nn.Sequential(
            nn.Conv3d(16, 32, kernel_size=(3, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(32), nn.ReLU(inplace=True))

        self.conv2d_1 = nn.Sequential(
            nn.Conv2d(96, 128, kernel_size=(3, 3), stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )

        self.conv2d_2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        self.fc1 = nn.Linear(43264, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, class_num)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        out = self.conv3d_1(x)
        out = self.conv3d_2(out)
        out = self.conv3d_3(out)
        # out = self.conv3d_4(out)
        out = self.conv2d_1(out.reshape(out.shape[0], -1, 17, 17))
        out = self.conv2d_2(out.reshape(out.shape[0], -1, 15, 15))

        out = out.reshape(out.shape[0], -1)
        out = F.relu(self.dropout(self.fc1(out)))
        out = F.relu(self.dropout(self.fc2(out)))
        out = self.fc3(out)
        return out

# 3. Data Preprocessing Modules

## 3.1 PCA for Data Dimensional Reducing

In [17]:
# 对高光谱数据 X 应用 PCA 变换
def applyPCA(X, numComponents):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
    return newX

## 3.2 Forming Patches of hyperspectral images

In [18]:
# 对单个像素周围提取 patch 时，边缘像素就无法取了，因此，给这部分像素进行 padding 操作
def padWithZeros(X, margin=2):
    newX = np.zeros(
        (X.shape[0] + 2 * margin, X.shape[1] + 2 * margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels=True):
    # 给 X 做 padding
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # 获得 y 中的标记样本数---10249
    count = 0
    for r in range(0, y.shape[0]):
        for c in range(0, y.shape[1]):
            if y[r, c] != 0:
                count = count + 1

    # split patches
    patchesData = np.zeros([count, windowSize, windowSize, X.shape[2]])
    patchesLabels = np.zeros(count)

    count = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            if y[r - margin, c - margin] != 0:
                patch = zeroPaddedX[r - margin:r + margin + 1,
                                    c - margin:c + margin + 1]
                patchesData[count, :, :, :] = patch
                patchesLabels[count] = y[r - margin, c - margin]
                count = count + 1

    return patchesData, patchesLabels

## 3.3 Splitting Dataset into trainSet & testingSet 

In [19]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=testRatio, random_state=randomState, stratify=y)
    return X_train, X_test, y_train, y_test

## 3.4 Data Loading

In [20]:
def loadData(name):
    data_path = "D:\VsCode WorkSpace\Hybrid2D&3D\Data"
    if name == 'IP':
        data = sio.loadmat(
            os.path.join(data_path, 'Indian-Pines\Indian_pines_corrected.mat')
        )['indian_pines_corrected']
        labels = sio.loadmat(
            os.path.join(
                data_path,
                'Indian-Pines\Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(
            os.path.join(data_path,
                         'Salinas\Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(
            data_path, 'Salinas\Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(
            os.path.join(data_path, 'Pavia-University\PaviaU.mat'))['paviaU']
        labels = sio.loadmat(
            os.path.join(data_path,
                         'Pavia-University\PaviaU_gt.mat'))['paviaU_gt']
    else:
        print("NO DATASET")
        exit()
    return data, labels

In [21]:
""" testing set """
class TestDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtest.shape[0]
        self.x_data = torch.FloatTensor(Xtest)
        self.y_data = torch.LongTensor(ytest)

    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len

""" training set """
class TrainDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtrain.shape[0]
        self.x_data = torch.FloatTensor(Xtrain)
        self.y_data = torch.LongTensor(ytrain)

    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len

# 4. Training Samples Initialization

In [22]:
name = "PU"
if (name == "IP" or name == "SA"):
    class_num = 16
elif (name == "PU"):
    class_num = 9

X, y = loadData(name)
# 用于测试样本的比例
test_ratio = 0.95
# 每个像素周围提取 patch 的尺寸
patch_size = 23
# 使用 PCA 降维，得到主成分的数量
pca_components = 15

print('Hyperspectral data shape: ', X.shape)  
print('Label shape: ', y.shape)  

print('\n... ... PCA tranformation ... ...')
X_pca = applyPCA(X, numComponents=pca_components)
print('Data shape after PCA: ', X_pca.shape)  

print('\n... ... create data cubes ... ...')
X_pca, y = createImageCubes(
    X_pca, y, windowSize=patch_size) 
print('Data cube X shape: ', X_pca.shape)  
print('Data cube y shape: ', y.shape)  

print('\n... ... create train & test data ... ...')
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X_pca, y, test_ratio)
print('Xtrain shape: ', Xtrain.shape)  
print('Xtest  shape: ', Xtest.shape)  

# 改变 Xtrain, Ytrain 的形状，以符合 keras 的要求
Xtrain = Xtrain.reshape(-1, patch_size, patch_size, pca_components, 1)
Xtest = Xtest.reshape(-1, patch_size, patch_size, pca_components, 1)
print('before transpose: Xtrain shape: ', Xtrain.shape)  
print('before transpose: Xtest  shape: ', Xtest.shape)  

# 为了适应 pytorch 结构，数据要做 transpose
Xtrain = Xtrain.transpose(0, 4, 3, 1, 2)
Xtest = Xtest.transpose(0, 4, 3, 1, 2)
print('after transpose: Xtrain shape: ', Xtrain.shape)  
print('after transpose: Xtest  shape: ', Xtest.shape)  
"""在此之前都是对数据的预处理"""
###########数据加载loader############
# 创建 trainloader 和 testloader
trainset = TrainDS()
testset = TestDS()
train_loader = torch.utils.data.DataLoader(
    dataset=trainset,
    batch_size=128,  # 128,53
    shuffle=True,
    num_workers=0)
test_loader = torch.utils.data.DataLoader(
    dataset=testset,
    batch_size=128,  # 128,53
    shuffle=False,
    num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
one = torch.ones(128, dtype=torch.long).to(device)
if test_ratio == 0.95:
    two = torch.ones(90, dtype=torch.long).to(device)
elif test_ratio == 0.90:
    two = torch.ones(53, dtype=torch.long).to(device)
elif test_ratio == 0.85:
    two = torch.ones(16, dtype=torch.long).to(device)

# 网络放到GPU上,一些metric的设定
net = FE_HybridSN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

Hyperspectral data shape:  (610, 340, 103)
Label shape:  (610, 340)

... ... PCA tranformation ... ...
Data shape after PCA:  (610, 340, 15)

... ... create data cubes ... ...
Data cube X shape:  (42776, 23, 23, 15)
Data cube y shape:  (42776,)

... ... create train & test data ... ...
Xtrain shape:  (2138, 23, 23, 15)
Xtest  shape:  (40638, 23, 23, 15)
before transpose: Xtrain shape:  (2138, 23, 23, 15, 1)
before transpose: Xtest  shape:  (40638, 23, 23, 15, 1)
after transpose: Xtrain shape:  (2138, 1, 15, 23, 23)
after transpose: Xtest  shape:  (40638, 1, 15, 23, 23)


# 5. Training

In [23]:
# 训练开始时间
start_time = time.time()

net.train()
total_loss = 0
# proc_bar = tqdm(range(150))
for epoch in range(150):
    # proc_bar.set_description(f"正处于第{epoch}回合：")
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        try:
            labels = labels - one
        except:
            labels = labels - two
        #print(labels)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('[Epoch: %d]   [loss avg: %.4f]   [current loss: %.4f]' %
          (epoch + 1, total_loss / (epoch + 1), loss.item()))

# proc_bar.close()

[Epoch: 1]   [loss avg: 24.8044]   [current loss: 0.6385]
[Epoch: 2]   [loss avg: 17.5055]   [current loss: 0.4460]
[Epoch: 3]   [loss avg: 13.8350]   [current loss: 0.2304]
[Epoch: 4]   [loss avg: 11.2554]   [current loss: 0.1105]
[Epoch: 5]   [loss avg: 9.5889]   [current loss: 0.2226]
[Epoch: 6]   [loss avg: 8.3239]   [current loss: 0.2013]
[Epoch: 7]   [loss avg: 7.3351]   [current loss: 0.1183]
[Epoch: 8]   [loss avg: 6.5868]   [current loss: 0.0783]
[Epoch: 9]   [loss avg: 5.9527]   [current loss: 0.1302]
[Epoch: 10]   [loss avg: 5.4374]   [current loss: 0.0206]
[Epoch: 11]   [loss avg: 5.0283]   [current loss: 0.0236]
[Epoch: 12]   [loss avg: 4.6780]   [current loss: 0.0064]
[Epoch: 13]   [loss avg: 4.3412]   [current loss: 0.0087]
[Epoch: 14]   [loss avg: 4.0433]   [current loss: 0.0020]
[Epoch: 15]   [loss avg: 3.7941]   [current loss: 0.0023]
[Epoch: 16]   [loss avg: 3.5839]   [current loss: 0.1348]
[Epoch: 17]   [loss avg: 3.3910]   [current loss: 0.0027]
[Epoch: 18]   [loss

## 5.1 Training time

In [24]:
end_time = time.time()
print(f"time cost:{(end_time-start_time)/60} min")
print('Finished Training')

time cost:8.103877933820089 min
Finished Training


# 6. Saving & Outputing the model params of proposed model

In [25]:
# 指定模型保存的地址
path = r'D:\VsCode WorkSpace\FEHN-FL\Assets\UP-Patch23\model.pth'
torch.save(net, path)
# 输出模型参数
summary(net, (1, 15, 23, 23))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1         [-1, 8, 9, 21, 21]             512
       BatchNorm3d-2         [-1, 8, 9, 21, 21]              16
              ReLU-3         [-1, 8, 9, 21, 21]               0
            Conv3d-4        [-1, 16, 5, 19, 19]           5,776
       BatchNorm3d-5        [-1, 16, 5, 19, 19]              32
              ReLU-6        [-1, 16, 5, 19, 19]               0
            Conv3d-7        [-1, 32, 3, 17, 17]          13,856
       BatchNorm3d-8        [-1, 32, 3, 17, 17]              64
              ReLU-9        [-1, 32, 3, 17, 17]               0
           Conv2d-10          [-1, 128, 15, 15]         110,720
      BatchNorm2d-11          [-1, 128, 15, 15]             256
             ReLU-12          [-1, 128, 15, 15]               0
           Conv2d-13          [-1, 256, 13, 13]         295,168
      BatchNorm2d-14          [-1, 256,

# 7. Evaluating the proposed model

In [26]:
a = np.ones(Xtest.shape[0])  ##9225是Xtest.shape[0]
ytest = ytest - a

count = 0
# 模型测试
net.eval()
for inputs, _ in test_loader:
    inputs = inputs.to(device)
    outputs = net(inputs)
    outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
    if count == 0:
        y_pred_test = outputs
        count = 1
    else:
        y_pred_test = np.concatenate((y_pred_test, outputs))

# 8. Generating the report of evaluation

## 8.1 text report

In [27]:
# 生成分类报告
classification = classification_report(ytest, y_pred_test, digits=4)
print(classification)


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc


def reports(test_loader, y_test, name):
    count = 0
    # 模型测试
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = net(inputs)
        outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
        if count == 0:
            y_pred = outputs
            count = 1
        else:
            y_pred = np.concatenate((y_pred, outputs))

    if name == 'IP':
        target_names = [
            'Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn', 'Grass-pasture',
            'Grass-trees', 'Grass-pasture-mowed', 'Hay-windrowed', 'Oats',
            'Soybean-notill', 'Soybean-mintill', 'Soybean-clean', 'Wheat',
            'Woods', 'Buildings-Grass-Trees-Drives', 'Stone-Steel-Towers'
        ]
    elif name == 'SA':
        target_names = [
            'Brocoli_green_weeds_1', 'Brocoli_green_weeds_2', 'Fallow',
            'Fallow_rough_plow', 'Fallow_smooth', 'Stubble', 'Celery',
            'Grapes_untrained', 'Soil_vinyard_develop',
            'Corn_senesced_green_weeds', 'Lettuce_romaine_4wk',
            'Lettuce_romaine_5wk', 'Lettuce_romaine_6wk',
            'Lettuce_romaine_7wk', 'Vinyard_untrained',
            'Vinyard_vertical_trellis'
        ]
    elif name == 'PU':
        target_names = [
            'Asphalt', 'Meadows', 'Gravel', 'Trees', 'Painted metal sheets',
            'Bare Soil', 'Bitumen', 'Self-Blocking Bricks', 'Shadows'
        ]

    classification = classification_report(y_test,
                                           y_pred,
                                           target_names=target_names)
    oa = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(y_test, y_pred)

    return classification, confusion, oa * 100, each_acc * 100, aa * 100, kappa * 100


# 将结果写在文件里
classification, confusion, oa, each_acc, aa, kappa = reports(
    test_loader, ytest, name)
classification = str(classification)
confusion = str(confusion)
file_name = r"D:\VsCode WorkSpace\FEHN-FL\Assets\UP-Patch23\classification_report(0.95).txt"

with open(file_name, 'w') as x_file:
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('{} training time (min)'.format((end_time-start_time)/60))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))

              precision    recall  f1-score   support

         0.0     0.9957    0.9925    0.9941      6299
         1.0     0.9994    0.9997    0.9995     17717
         2.0     0.9861    0.9995    0.9928      1994
         3.0     0.9972    0.9928    0.9950      2911
         4.0     1.0000    0.9867    0.9933      1278
         5.0     0.9998    1.0000    0.9999      4778
         6.0     0.9976    0.9952    0.9964      1263
         7.0     0.9833    0.9920    0.9876      3498
         8.0     0.9730    0.9600    0.9664       900

    accuracy                         0.9960     40638
   macro avg     0.9925    0.9909    0.9917     40638
weighted avg     0.9960    0.9960    0.9960     40638



## 8.2 images of prediction report

In [28]:
# 显示结果
# load the original image

# X, y = loadData(name)

# height = y.shape[0]
# width = y.shape[1]

# X = applyPCA(X, numComponents=pca_components)
# X = padWithZeros(X, patch_size // 2)

# # 逐像素预测类别
# outputs = np.zeros((height, width))
# for i in range(height):
#     for j in range(width):
#         if int(y[i, j]) == 0:
#             continue
#         else:
#             image_patch = X[i:i + patch_size, j:j + patch_size, :]
#             image_patch = image_patch.reshape(1, image_patch.shape[0],
#                                               image_patch.shape[1],
#                                               image_patch.shape[2], 1)
#             X_test_image = torch.FloatTensor(
#                 image_patch.transpose(0, 4, 3, 1, 2)).to(device)
#             prediction = net(X_test_image)
#             prediction = np.argmax(prediction.detach().cpu().numpy(), axis=1)
#             outputs[i][j] = prediction + 1
#     if i % 20 == 0:
#         print('... ... row ', i, ' handling ... ...')

# oringal_image = spectral.imshow(classes=y, figsize=(7, 7))
# predict_image = spectral.imshow(classes=outputs.astype(int), figsize=(7, 7))
# spectral.save_rgb("Assets\FE-NET\(PU-3-3-0.85)FE-NET-原始.jpg",
#                   y.astype(int),
#                   colors=spectral.spy_colors)
# spectral.save_rgb("Assets\FE-NET\(PU-3-3-0.85)FE-NET-预测.jpg",
#                   outputs.astype(int),
#                   colors=spectral.spy_colors)